In [1]:
import h5py
import numpy as np
import random
import pdb
from generator_helper_functions import create_user_maxtrix, create_user_ids, latent_user_input_layer

f = h5py.File('../data/Full_NF.h5', 'r')
dset = f['dataset_1']
dset = dset[:]
print("Data loaded from file")

dset_indicies = set(range(len(dset)))
train_num = int(.9 * len(dset))
print("Training Samples: ", train_num, "| Validation Samples: ", len(dset) - train_num)

user_ids = create_user_ids(dset)
train_indicies = random.sample(dset_indicies, train_num)
validation_indicies = dset_indicies.difference(train_indicies)
training_data = np.array([dset[i] for i in train_indicies])
del train_indicies
validation_data = np.array([dset[i] for i in validation_indicies])
del validation_indicies
print("Training and Validation datasets separated")
#del dset

/home/steich/NF_Prize_Data/nf_prize_env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Data loaded from file
Training Samples:  90432456 | Validation Samples:  10048051
Training and Validation datasets separated


In [2]:
user_ids = create_user_ids(dset)
k = 50

In [3]:
#train_indices[0]

In [4]:
del dset_indicies
#del train_indicies
#del validation_indicies
del dset

In [5]:
user_matrix = create_user_maxtrix(training_data, user_ids, k_n=k)

In [6]:
import pickle

In [7]:
with open('../data/user_matrix.pickle', 'wb') as handle:
    pickle.dump(user_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../data/training_data.pickle', 'wb') as handle:
    pickle.dump(training_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../data/validation_data.pickle', 'wb') as handle:
    pickle.dump(validation_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../data/user_ids.pickle', 'wb') as handle:
    pickle.dump(user_ids, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
del user_matrix
del training_data
del validation_data
del user_ids

In [1]:
import pickle
with open('../data/user_matrix.pickle', 'rb') as handle:
    user_matrix = pickle.load(handle)
with open('../data/training_data.pickle', 'rb') as handle:
    training_data = pickle.load(handle)
with open('../data/validation_data.pickle', 'rb') as handle:
    validation_data = pickle.load(handle)
with open('../data/user_ids.pickle', 'rb') as handle:
    user_ids = pickle.load(handle)

In [2]:
from sklearn.preprocessing import MinMaxScaler

In [3]:
transformer = MinMaxScaler(copy=False)
transformer.fit_transform(user_matrix)
user_matrix

array([[0.57182926, 0.60183805, 0.45889142, ..., 0.7786627 , 0.73003244,
        0.97921675],
       [0.5277477 , 0.53475094, 0.4299253 , ..., 0.87229574, 0.43106568,
        0.67818606],
       [0.5079858 , 0.66573393, 0.50564766, ..., 0.7727094 , 0.70960677,
        0.7534699 ],
       ...,
       [0.5052166 , 0.5821171 , 0.4181541 , ..., 0.7926306 , 0.77288043,
        0.9436375 ],
       [0.56737584, 0.62411636, 0.46938246, ..., 0.7722611 , 0.69581765,
        0.99700797],
       [0.56497455, 0.62441105, 0.48783594, ..., 0.77094686, 0.70339984,
        0.98711175]], dtype=float32)

In [4]:
k = 50      
def nf_label_transformation_function(label):
    
    one_hot = np.zeros(5)
    one_hot[label - 1] = 1
    return one_hot

import warnings
warnings.simplefilter('ignore')

import keras
#import random
import numpy as np
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from generator_helper_functions import  latent_user_input_layer, Generate_User_Latent_Feedforward

#opt = keras.optimizers.rmsprop()
opt = keras.optimizers.SGD()


# k dim user vector + number of movies + statistics + month, day, year
input_size = k + 17770

batch_size = 64

model = Sequential()
model.add(Dense(input_size, activation='relu', input_shape=(input_size,)))
model.add(Dense(1000, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer='sgd',
             metrics=['mse', 'mae', 'acc'])

print("Model Compiled")

Using TensorFlow backend.


Model Compiled


In [5]:
def nf_label_transformation_function(label):
    
    one_hot = np.zeros(5)
    one_hot[label - 1] = 1
    return one_hot

train_gen = Generate_User_Latent_Feedforward(training_data, 
                                batch_size, 
                                (input_size,), 
                                (5,), #label shape
                                latent_user_input_layer, 
                                lambda x: nf_label_transformation_function(x[2]),
                                user_matrix,
                                user_ids,
                                normalized = True
                                )

validation_gen = Generate_User_Latent_Feedforward(validation_data, 
                                batch_size, 
                                (input_size,), 
                                (5,), #label shape
                                latent_user_input_layer, 
                                lambda x: nf_label_transformation_function(x[2]),
                                user_matrix,
                                user_ids,
                                normalized = True
                                )

checkpointer = ModelCheckpoint(filepath='../models/latent_user_SGD_MSLE_K50_cat_v1__normalized.hdf5', verbose=1, save_best_only=True)
model.fit_generator(train_gen,
                    epochs=4,
                    steps_per_epoch=1000,
                    validation_data=validation_gen,
                    validation_steps=100,
                    callbacks=[checkpointer]
                    )

Epoch 1/4
1000/1000 [==============================] - 1900s 2s/step - loss: 1.4391 - mean_squared_error: 0.1481 - mean_absolute_error: 0.2966 - acc: 0.3355 - val_loss: 1.4384 - val_mean_squared_error: 0.1479 - val_mean_absolute_error: 0.2960 - val_acc: 0.3425

Epoch 00001: val_loss improved from inf to 1.43835, saving model to ../models/latent_user_SGD_MSLE_K50_cat_v1__normalized.hdf5
Epoch 2/4
1000/1000 [==============================] - 5674s 6s/step - loss: 1.4368 - mean_squared_error: 0.1479 - mean_absolute_error: 0.2958 - acc: 0.3378 - val_loss: 1.4282 - val_mean_squared_error: 0.1473 - val_mean_absolute_error: 0.2946 - val_acc: 0.3384

Epoch 00002: val_loss improved from 1.43835 to 1.42821, saving model to ../models/latent_user_SGD_MSLE_K50_cat_v1__normalized.hdf5
Epoch 3/4
1000/1000 [==============================] - 1884s 2s/step - loss: 1.4359 - mean_squared_error: 0.1479 - mean_absolute_error: 0.2959 - acc: 0.3340 - val_loss: 1.4187 - val_mean_squared_error: 0.1468 - val_mea

In [ ]:
validation_data[1]

array([    6042,   173633,        4, 20051106])

In [ ]:
from sklearn.metrics import precision_score

In [ ]:
precision_score(
    list(map(lambda x: round(float(model.predict(np.expand_dims(latent_user_input_layer(x, user_matrix, user_ids, True), axis=0)) * 5)), 
             validation_data[0:3000])), 
    validation_data[0:3000,2], average='micro') 



In [ ]:
validation_data[0:3, 2]

In [ ]:
user_ids = {user:i for i, user in enumerate(user_ids)}

In [ ]:
user_rating_matrix

In [ ]:
user_rating_matrix